# Testes

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Em <<_TheEuroProblem>> apresentei um problema do livro de David MacKay, [*Information Theory, Inference, and Learning Algorithms*](http://www.inference.org.uk/mackay/itila/p0.html):

"Uma declaração estatística apareceu em *The Guardian* na sexta-feira 4 de Janeiro de 2002:

> Quando fiada no bordo 250 vezes, uma moeda belga de um euro veio à cabeça 140 vezes e cauda 110.  \Parece-me muito suspeito", disse Barry Blight, um professor de estatística da London School of Economics.  \Se a moeda fosse imparcial, a hipótese de obter um resultado tão extremo como esse seria inferior a 7%".

"Mas [MacKay pergunta] será que estes dados dão provas de que a moeda é tendenciosa em vez de justa?"

Começámos a responder a esta pergunta em <<_EstimatingProportions>>; para rever, a nossa resposta baseou-se nestas decisões de modelação:

* Se se girar uma moeda no bordo, há alguma probabilidade, $x$, de que ela caia de cabeça.

* O valor de $x$ varia de uma moeda para outra, dependendo de como a moeda é equilibrada e possivelmente de outros factores.

Começando com uma distribuição prévia uniforme por $x$, actualizámo-la com os dados fornecidos, 140 cabeças e 110 caudas.  Depois utilizámos a distribuição posterior para calcular o valor mais provável de $x$, a média posterior, e um intervalo credível.

Mas nunca respondemos realmente à pergunta de MacKay: "Será que estes dados dão provas de que a moeda é tendenciosa em vez de justa?"

Neste capítulo, por fim, vamos fazê-lo.

## Estimativa

Vamos rever a solução para o problema do Euro de <<_TheBinomialLikelihoodFunction>>>.  Começámos com um anterior uniforme.

In [4]:
import numpy as np
from empiricaldist import Pmf

xs = np.linspace(0, 1, 101)
uniform = Pmf(1, xs)

E utilizámos a distribuição binomial para calcular a probabilidade dos dados para cada valor possível de $x$.

In [5]:
from scipy.stats import binom

k, n = 140, 250
likelihood = binom.pmf(k, n, xs)

Calculámos a distribuição posterior da forma habitual.

In [6]:
posterior = uniform * likelihood
posterior.normalize()

E aqui está o que parece.

In [7]:
from utils import decorate

posterior.plot(label='140 heads out of 250')

decorate(xlabel='Proportion of heads (x)',
         ylabel='Probability',
         title='Posterior distribution of x')

Mais uma vez, a média posterior é cerca de 0,56, com um intervalo credível de 90% de 0,51 a 0,61.

In [8]:
print(posterior.mean(), 
      posterior.credible_interval(0.9))

A média anterior era de 0,5, e a média posterior é de 0,56, pelo que parece que os dados são prova de que a moeda é tendenciosa.

Mas, afinal, não é assim tão simples.

## Evidência

Em <<_OliversBlood>>, eu disse que os dados são considerados evidência a favor de uma hipótese, $A$, se os dados são mais prováveis abaixo de $A$ do que abaixo da alternativa, $B$; isto é, se

$$P(D|A) > P(D|B)$$

Além disso, podemos quantificar a força das provas calculando o rácio destas probabilidades, que é conhecido como o [Bayes factor](https://en.wikipedia.org/wiki/Bayes_factor) e muitas vezes denotado $K$:

$$K = \frac{P(D|A)}{P(D|B)}$$$

Assim, para o problema do Euro, consideremos duas hipóteses, "feir" e "biased", e calculemos a probabilidade dos dados sob cada hipótese.

Se a moeda for justa, a probabilidade de cabeças é de 50%, e podemos calcular a probabilidade dos dados (140 cabeças em 250 rotações) utilizando a distribuição binomial:

In [9]:
k = 140
n = 250

like_fair = binom.pmf(k, n, p=0.5)
like_fair

Essa é a probabilidade dos dados, dado que a moeda é justa.

Mas se a moeda é tendenciosa, qual é a probabilidade dos dados?  Isso depende do que significa "enviesado".

Se soubermos antecipadamente que "enviesado" significa que a probabilidade de cabeças é de 56%, podemos utilizar novamente a distribuição binomial:

In [10]:
like_biased = binom.pmf(k, n, p=0.56)
like_biased

Agora podemos calcular o rácio de probabilidade:

In [11]:
K = like_biased / like_fair
K

Os dados são cerca de 6 vezes mais prováveis se a moeda for tendenciosa, por esta definição, do que se for justa.

Mas utilizámos os dados para definir a hipótese, o que parece ser batota.  Para ser justo, devemos definir "enviesado" antes de vermos os dados.

## Viés de Distribuição Uniforme

Suponha que "parcial" significa que a probabilidade de cabeças é tudo menos 50%, e todos os outros valores são igualmente prováveis.

Podemos representar essa definição, fazendo uma distribuição uniforme e retirando 50%.

In [12]:
biased_uniform = uniform.copy()
biased_uniform[0.5] = 0
biased_uniform.normalize()

Para calcular a probabilidade total dos dados sob esta hipótese, calculamos a probabilidade condicional dos dados para cada valor de $x$.

In [13]:
xs = biased_uniform.qs
likelihood = binom.pmf(k, n, xs)

Depois multiplicar pelas probabilidades anteriores e somar os produtos:

In [14]:
like_uniform = np.sum(biased_uniform * likelihood)
like_uniform

Esta é a probabilidade dos dados sob a hipótese do "uniforme tendencioso".

Agora podemos calcular o rácio de probabilidade dos dados sob as hipóteses de "feira" e "uniforme":

In [15]:
K = like_fair / like_uniform
K

Os dados são cerca de duas vezes mais prováveis se a moeda for justa do que se for enviesada, por esta definição de "enviesada".

Para ter uma noção da força dessa prova, podemos aplicar a regra de Bayes.

Por exemplo, se a probabilidade anterior é de 50% de que a moeda seja tendenciosa, as probabilidades anteriores são de 1, portanto as probabilidades posteriores são de cerca de 2,1 para 1 e a probabilidade posterior é de cerca de 68%.

In [16]:
prior_odds = 1
posterior_odds = prior_odds * K
posterior_odds

In [17]:
def prob(o):
    return o / (o+1)

In [18]:
posterior_probability = prob(posterior_odds)
posterior_probability

A prova de que "move a agulha" de 50% para 68% não é muito forte.

Agora suponha que "enviesado" não significa que cada valor de $x$ seja igualmente provável.  Talvez valores próximos dos 50% sejam mais prováveis e valores próximos dos extremos sejam menos prováveis.

Poderíamos utilizar uma distribuição em forma de triângulo para representar esta definição alternativa de "enviesado":

In [19]:
ramp_up = np.arange(50)
ramp_down = np.arange(50, -1, -1)
a = np.append(ramp_up, ramp_down)

triangle = Pmf(a, xs, name='triangle')
triangle.normalize()

Tal como fizemos com a distribuição uniforme, podemos remover 50% como valor possível de $x$ (mas não faz muita diferença se saltarmos este detalhe).

In [20]:
biased_triangle = triangle.copy()
biased_triangle[0.5] = 0
biased_triangle.normalize()

Eis como se parece o triângulo anterior, em comparação com o anterior uniforme.

In [21]:
biased_uniform.plot(label='uniform prior')
biased_triangle.plot(label='triangle prior')

decorate(xlabel='Proportion of heads (x)',
         ylabel='Probability',
         title='Uniform and triangle prior distributions')

**Exercício:** Agora calcula a probabilidade total dos dados sob esta definição de "enviesado" e calcula o factor Bayes, em comparação com a hipótese justa.

Os dados são prova de que a moeda é tendenciosa?

In [22]:
# Solution goes here

In [23]:
# Solution goes here

In [24]:
# Solution goes here

## Teste de Hipótese Bayesiana

O que fizemos até agora neste capítulo é por vezes chamado "teste de hipóteses Bayesianas" em contraste com [statistical hypothesis testing](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing).

No teste de hipóteses estatísticas, calculamos um valor p, que é difícil de definir de forma concisa, e usamo-lo para determinar se os resultados são "estatisticamente significativos", o que também é difícil de definir de forma concisa.

A alternativa Bayesiana é relatar o factor Bayes, $K$, que resume a força da evidência a favor de uma hipótese ou de outra.

Algumas pessoas pensam que é melhor reportar $K$ do que uma probabilidade posterior porque $K$ não depende de uma probabilidade anterior.

Mas como vimos neste exemplo, $K$ depende frequentemente de uma definição precisa das hipóteses, o que pode ser tão controverso como uma probabilidade anterior.

Na minha opinião, o teste de hipóteses Bayesiano é melhor porque mede a força das provas num continuum, em vez de tentar fazer uma determinação binária.

Mas não resolve o que eu penso ser o problema fundamental, que é que o teste de hipóteses não é colocar a questão que realmente nos interessa.

Para ver porquê, suponha que testa a moeda e decide que afinal é tendenciosa.  O que pode fazer com esta resposta?  Na minha opinião, não muito.

Em contraste, há duas questões que penso serem mais úteis (e por isso mais significativas):

* Predição: Com base no que sabemos sobre a moeda, o que devemos esperar que aconteça no futuro?

* Tomada de decisões: Podemos utilizar essas previsões para tomar melhores decisões?

Neste momento, vimos alguns exemplos de previsão.  Por exemplo, em <<_PoissonProcessos>> utilizámos a distribuição posterior das taxas de marcação de golos para prever o resultado dos jogos de futebol.

E já vimos um exemplo anterior de análise de decisão: Em <<_DecisionAnalysis>> utilizámos a distribuição de preços para escolher uma oferta óptima em *The Price is Right*.

Portanto, vamos terminar este capítulo com outro exemplo de análise de decisão Bayesiana, a estratégia do Bandido Bayesiano.

## Bandidos Bayesianos

Se alguma vez esteve num casino, provavelmente já viu uma slot machine, que por vezes é chamada de "bandido de um só braço" porque tem uma pega como um braço e a capacidade de levar dinheiro como um bandido.

A estratégia do Bandido Bayesiano tem o nome de bandido de um braço porque resolve um problema com base numa versão simplificada de uma slot machine.

Suponha que cada vez que jogar numa slot machine, há uma probabilidade fixa de ganhar.  E suponha que máquinas diferentes lhe dão diferentes probabilidades de ganhar, mas não sabe quais são as probabilidades.

Inicialmente, tem a mesma crença prévia sobre cada uma das máquinas, pelo que não tem razões para preferir uma em detrimento das outras.  Mas se tocar em cada máquina algumas vezes, pode utilizar os resultados para estimar as probabilidades.  E pode utilizar as probabilidades estimadas para decidir qual a máquina a jogar a seguir.

A um nível elevado, essa é a estratégia do bandido Bayesiano.  Agora vamos ver os detalhes.

## Crenças anteriores

Se não sabemos nada sobre a probabilidade de ganhar, podemos começar com um anterior uniforme.

In [25]:
xs = np.linspace(0, 1, 101)
prior = Pmf(1, xs)
prior.normalize()

Supondo que estamos a escolher entre quatro slot machines, farei quatro cópias do anterior, uma para cada máquina.

In [26]:
beliefs = [prior.copy() for i in range(4)]

Esta função apresenta quatro distribuições numa grelha.

In [27]:
import matplotlib.pyplot as plt

options = dict(xticklabels='invisible', yticklabels='invisible')

def plot(beliefs, **options):
    for i, pmf in enumerate(beliefs):
        plt.subplot(2, 2, i+1)
        pmf.plot(label='Machine %s' % i)
        decorate(yticklabels=[])
        
        if i in [0, 2]:
            decorate(ylabel='PDF')
        
        if i in [2, 3]:
            decorate(xlabel='Probability of winning')
        
    plt.tight_layout()

Eis como são as distribuições anteriores para as quatro máquinas.

In [28]:
plot(beliefs)

## A Actualização

Cada vez que tocamos uma máquina, podemos usar o resultado para actualizar as nossas crenças.  A função seguinte faz a actualização.

In [29]:
likelihood = {
    'W': xs,
    'L': 1 - xs
}

In [30]:
def update(pmf, data):
    """Update the probability of winning."""
    pmf *= likelihood[data]
    pmf.normalize()

Esta função actualiza a distribuição prévia em vigor.

`pmf` é um `Pmf` que representa a distribuição prévia de `x`, que é a probabilidade de ganhar.

"Dados" é uma cadeia, ou "W" se o resultado for um ganho ou "L" se o resultado for uma perda.

A probabilidade dos dados é de `xs` ou `1-xs`, dependendo do resultado.

Suponhamos que escolhemos uma máquina, jogamos 10 vezes, e ganhamos uma vez.  Podemos calcular a distribuição posterior de `x`, com base neste resultado, desta forma:

In [31]:
np.random.seed(17)

In [32]:
bandit = prior.copy()

for outcome in 'WLLLLLLLLL':
    update(bandit, outcome)

Eis como se parece o posterior.

In [33]:
bandit.plot()
decorate(xlabel='Probability of winning',
         ylabel='PDF',
         title='Posterior distribution, nine losses, one win')

## Bandidos Múltiplos

Agora suponha-se que temos quatro máquinas com estas probabilidades:

In [34]:
actual_probs = [0.10, 0.20, 0.30, 0.40]

Lembre-se que, como jogador, não conhecemos estas probabilidades.

A função seguinte toma o índice de uma máquina, simula tocar a máquina uma vez, e devolve o resultado, `W` ou `L`.

In [35]:
from collections import Counter

# count how many times we've played each machine
counter = Counter()

def play(i):
    """Play machine i.
    
    i: index of the machine to play
    
    returns: string 'W' or 'L'
    """
    counter[i] += 1
    p = actual_probs[i]
    if np.random.random() < p:
        return 'W'
    else:
        return 'L'

"Contador" é um "Contador", que é uma espécie de dicionário que utilizaremos para controlar quantas vezes cada máquina é tocada.

Aqui está um teste que toca cada máquina 10 vezes.

In [36]:
for i in range(4):
    for _ in range(10):
        outcome = play(i)
        update(beliefs[i], outcome)

Cada vez através do laço interior, jogamos uma máquina e actualizamos as nossas crenças.

Eis como são as nossas crenças posteriores.

In [37]:
plot(beliefs)

Aqui estão as probabilidades reais, meios posteriores, e intervalos credíveis de 90%.

In [38]:
import pandas as pd

def summarize_beliefs(beliefs):
    """Compute means and credible intervals.
    
    beliefs: sequence of Pmf
    
    returns: DataFrame
    """
    columns = ['Actual P(win)', 
               'Posterior mean', 
               'Credible interval']
    
    df = pd.DataFrame(columns=columns)
    for i, b in enumerate(beliefs):
        mean = np.round(b.mean(), 3)
        ci = b.credible_interval(0.9)
        ci = np.round(ci, 3)
        df.loc[i] = actual_probs[i], mean, ci
    return df

In [39]:
summarize_beliefs(beliefs)

Esperamos que os intervalos credíveis contenham as probabilidades reais a maior parte do tempo.

## Explorar e Explorar

Com base nestas distribuições posteriores, que máquina acha que devemos jogar a seguir?  Uma opção seria escolher a máquina com a maior média posterior.  

Isso não seria uma má ideia, mas tem um inconveniente: uma vez que só tocámos cada máquina algumas vezes, as distribuições posteriores são amplas e sobrepostas, o que significa que não temos a certeza de qual é a melhor máquina; se nos concentrarmos numa máquina demasiado cedo, podemos escolher a máquina errada e tocá-la mais do que deveríamos.

Para evitar esse problema, poderíamos ir para o outro extremo e jogar todas as máquinas igualmente até estarmos confiantes de que identificámos a melhor máquina, e depois jogá-la exclusivamente.

Também não é uma má ideia, mas tem um inconveniente: enquanto estamos a recolher dados, não estamos a fazer bom uso deles; enquanto não tivermos a certeza de qual é a melhor máquina, estamos a jogar as outras mais do que deveríamos.

A estratégia dos Bandidos Bayesianos evita ambos os inconvenientes ao recolher e utilizar dados ao mesmo tempo.  Por outras palavras, equilibra a exploração e a exploração.

O núcleo da ideia chama-se [Thompson sampling](https://en.wikipedia.org/wiki/Thompson_sampling): quando escolhemos uma máquina, escolhemos ao acaso de modo a que a probabilidade de escolher cada máquina seja proporcional à probabilidade de ser a melhor.

Dadas as distribuições posteriores, podemos calcular a "probabilidade de superioridade" para cada máquina.

Aqui está uma maneira de o fazer.  Podemos retirar uma amostra de 1000 valores de cada distribuição posterior, como esta:

In [40]:
samples = np.array([b.choice(1000) 
                    for b in beliefs])
samples.shape

O resultado tem 4 filas e 1000 colunas.  Podemos utilizar `argmax` para encontrar o índice do maior valor em cada coluna:

In [41]:
indices = np.argmax(samples, axis=0)
indices.shape

O `Pmf` destes índices é a fracção de vezes que cada máquina produziu os valores mais elevados.

In [42]:
pmf = Pmf.from_seq(indices)
pmf

Estas fracções aproximam-se da probabilidade de superioridade para cada máquina.  Assim, poderíamos escolher a máquina seguinte, escolhendo um valor deste `Pmf`.

In [43]:
pmf.choice()

Mas isso é muito trabalho para escolher um único valor, e não é realmente necessário, porque há um atalho.

Se retirarmos um único valor aleatório de cada distribuição posterior e seleccionarmos a máquina que produz o valor mais elevado, verifica-se que seleccionaremos cada máquina em proporção à sua probabilidade de superioridade.

É isso que a seguinte função faz.

In [44]:
def choose(beliefs):
    """Use Thompson sampling to choose a machine.
    
    Draws a single sample from each distribution.
    
    returns: index of the machine that yielded the highest value
    """
    ps = [b.choice() for b in beliefs]
    return np.argmax(ps)

Esta função escolhe um valor da distribuição posterior de cada máquina e depois utiliza "argmax" para encontrar o índice da máquina que produziu o valor mais alto.

Aqui está um exemplo.

In [45]:
choose(beliefs)

## A Estratégia

Juntando tudo isto, a seguinte função escolhe uma máquina, joga uma vez, e actualiza `crenças':

In [46]:
def choose_play_update(beliefs):
    """Choose a machine, play it, and update beliefs."""
    
    # choose a machine
    machine = choose(beliefs)
    
    # play it
    outcome = play(machine)
    
    # update beliefs
    update(beliefs[machine], outcome)

Para o testar, comecemos de novo com um novo conjunto de crenças e um "balcão" vazio.

In [47]:
beliefs = [prior.copy() for i in range(4)]
counter = Counter()

Se executarmos o algoritmo do bandido 100 vezes, podemos ver como "a crença" é actualizada:

In [48]:
num_plays = 100

for i in range(num_plays):
    choose_play_update(beliefs)
    
plot(beliefs)

O quadro seguinte resume os resultados.

In [49]:
summarize_beliefs(beliefs)

Os intervalos credíveis contêm geralmente as probabilidades reais de vencer.

As estimativas ainda são aproximadas, especialmente para as máquinas de menor probabilidade.  Mas isso é uma característica, não um bug: o objectivo é jogar mais frequentemente com as máquinas de alta-probabilidade.  Tornar as estimativas mais precisas é um meio para esse fim, mas não um fim em si.

Mais importante ainda, vamos ver quantas vezes cada máquina foi tocada.  

In [50]:
def summarize_counter(counter):
    """Report the number of times each machine was played.
    
    counter: Collections.Counter
    
    returns: DataFrame
    """
    index = range(4)
    columns = ['Actual P(win)', 'Times played']
    df = pd.DataFrame(index=index, columns=columns)
    for i, count in counter.items():
        df.loc[i] = actual_probs[i], count
    return df

In [51]:
summarize_counter(counter)

Se as coisas correrem conforme o planeado, as máquinas com probabilidades mais elevadas devem ser jogadas com mais frequência.

## Resumo

Neste capítulo finalmente resolvemos o problema do Euro, determinando se os dados apoiam a hipótese de que a moeda é justa ou tendenciosa.  Descobrimos que a resposta depende de como definimos "enviesada".  E resumimos os resultados utilizando um factor Bayes, que quantifica a força da evidência.

Mas a resposta não foi satisfatória porque, na minha opinião, a pergunta não era interessante.  Saber se a moeda é tendenciosa não é útil, a menos que nos ajude a fazer melhores previsões e melhores decisões.

Como exemplo de uma questão mais interessante, analisámos o problema do "bandido de um só braço" e uma estratégia para o resolver, o algoritmo do bandido Bayesiano, que tenta equilibrar exploração e exploração, ou seja, recolher mais informação e fazer o melhor uso da informação de que dispomos.

Como exercício, terá a oportunidade de explorar estratégias adaptativas para testes padronizados.

Os bandidos Bayesianos e os testes adaptativos são exemplos de [Bayesian decision theory](https://wiki.lesswrong.com/wiki/Bayesian_decision_theory), que é a ideia de utilizar uma distribuição posterior como parte de um processo de tomada de decisão, muitas vezes escolhendo uma acção que minimize os custos que esperamos em média (ou maximize um benefício).

A estratégia que utilizámos em <<_MaximizingExpectedGain>> para licitar em *O Preço Está Certo* é outro exemplo.

Estas estratégias demonstram o que eu penso ser a maior vantagem dos métodos Bayesianos em relação às estatísticas clássicas.  Quando representamos o conhecimento na forma de distribuições de probabilidade, o teorema de Bayes diz-nos como mudar as nossas crenças à medida que obtemos mais dados, e a teoria de decisão Bayesiana diz-nos como tornar esse conhecimento accionável.

## Exercícios

**Exercício:** Testes padronizados como o [SAT](https://en.wikipedia.org/wiki/SAT) são frequentemente utilizados como parte do processo de admissão nas faculdades e universidades.

O objectivo do SAT é medir a preparação académica dos testadores; se for preciso, a sua pontuação deve reflectir a sua capacidade real no domínio do teste.

Até recentemente, testes como o SAT eram feitos com papel e lápis, mas agora os estudantes têm a opção de fazer o teste online.  No formato online, é possível que o teste seja "adaptável", o que significa que pode [choose each question based on responses to previous questions](https://www.nytimes.com/2018/04/05/education/learning/tests-act-sat.html).

Se um aluno acertar nas primeiras perguntas, o teste pode desafiá-lo com perguntas mais difíceis.  Se estiverem a debater-se, pode dar-lhes perguntas mais fáceis.

O teste adaptativo tem o potencial de ser mais "eficiente", o que significa que com o mesmo número de perguntas um teste adaptativo poderia medir com maior precisão a capacidade de um testador.

Para ver se isto é verdade, vamos desenvolver um modelo de teste adaptativo e quantificar a precisão das suas medições.

Os detalhes deste exercício encontram-se no caderno de notas.

## O Modelo

O modelo que vamos utilizar baseia-se no [item response theory](https://en.wikipedia.org/wiki/Item_response_theory), que pressupõe que podemos quantificar a dificuldade de cada pergunta e a capacidade de cada test-taker, e que a probabilidade de uma resposta correcta é uma função da dificuldade e da capacidade.

Especificamente, um pressuposto comum é que esta função é uma função logística de três parâmetros:

$$$mathrm{p} = c + \frac{1-c}{1 + e^{-a (theta-b)}$$$

onde $$$ é a capacidade do test-taker e $b$ é a dificuldade da pergunta.

$c$ é a probabilidade mais baixa de obter uma pergunta correcta, supondo que o test-taker com a menor capacidade tenta responder à pergunta mais difícil.  Num teste de escolha múltipla com quatro respostas, $c$ pode ser 0,25, que é a probabilidade de obter a resposta certa ao adivinhar ao acaso.

$a$ controla a forma da curva.

A função seguinte calcula a probabilidade de uma resposta correcta, dada a "capacidade" e a "dificuldade":

In [52]:
def prob_correct(ability, difficulty):
    """Probability of a correct response."""
    a = 100
    c = 0.25
    x = (ability - difficulty) / a
    p = c + (1-c) / (1 + np.exp(-x))
    return p

Escolhi `a' para tornar a gama de pontuações comparável ao SAT, que reporta pontuações de 200 a 800.

Eis como se apresenta a curva logística para uma pergunta com dificuldade 500 e uma gama de capacidades.

In [53]:
abilities = np.linspace(100, 900)
diff = 500
ps = prob_correct(abilities, diff)

In [54]:
plt.plot(abilities, ps)
decorate(xlabel='Ability',
         ylabel='Probability correct',
         title='Probability of correct answer, difficulty=500',
         ylim=[0, 1.05])

Alguém com "capacidade=900" tem quase a certeza de obter a resposta certa.

Alguém com "capacidade=100" tem cerca de 25% de mudança em obter a resposta certa por adivinhação.

## Simulando o Teste

Para simular o teste, utilizaremos a mesma estrutura que utilizámos para a estratégia do bandido:

* Uma função chamada `play` que simula um testador respondendo a uma pergunta.

* Uma função chamada "escolha" que escolhe a próxima pergunta a colocar.

* Uma função chamada "actualização" que utiliza o resultado (uma resposta correcta ou não) para actualizar a estimativa da capacidade do test-taker.

Aqui está o "jogo", que toma como parâmetros a "capacidade" e a "dificuldade".

In [55]:
def play(ability, difficulty):
    """Simulate a test-taker answering a question."""
    p = prob_correct(ability, difficulty)
    return np.random.random() < p

O valor de retorno é "Verdadeiro" para uma resposta correcta e "Falso" para um valor aleatório entre 0 e 1.

Como teste, vamos simular um testador com "capacidade=600" a responder a uma pergunta com "dificuldade=500".  A probabilidade de uma resposta correcta é de cerca de 80%.

In [56]:
prob_correct(600, 500)

Suponha que esta pessoa faz um teste com 51 perguntas, todas com a mesma dificuldade, `500`.

Esperamos que eles respondam a cerca de 80% das perguntas.

Aqui está o resultado de uma simulação.

In [57]:
np.random.seed(18)

In [58]:
num_questions = 51
outcomes = [play(600, 500) for _ in range(num_questions)]
np.mean(outcomes)

Esperamos que acertem cerca de 80% das perguntas.

Agora vamos supor que não conhecemos a capacidade do test-taker.  Podemos utilizar os dados que acabámos de gerar para os estimar.

E é isso que vamos fazer a seguir.

## O Prior

O SAT foi concebido para que a distribuição das pontuações seja aproximadamente normal, com média 500 e desvio padrão 100.

Assim, a pontuação mais baixa, 200, é três desvios padrão abaixo da média, e a pontuação mais alta, 800, é três desvios padrão acima.

Poderíamos utilizar essa distribuição como uma distribuição prévia, mas tenderia a cortar os extremos baixo e alto da distribuição.

Em vez disso, vou inflar o desvio padrão para 300, para deixar em aberto a possibilidade de "capacidade" poder ser inferior a 200 ou superior a 800.

Aqui está um `Pmf` que representa a distribuição prévia.

In [59]:
from scipy.stats import norm

mean = 500
std = 300

qs = np.linspace(0, 1000)
ps = norm(mean, std).pdf(qs)

prior = Pmf(ps, qs)
prior.normalize()

E aqui está o que parece.

In [60]:
prior.plot(label='std=300', color='C5')

decorate(xlabel='Ability',
         ylabel='PDF',
         title='Prior distribution of ability',
         ylim=[0, 0.032])

## A Actualização

A função seguinte toma um `Pmf` prévio e o resultado de uma única pergunta, e actualiza o `Pmf` no lugar.

In [61]:
def update_ability(pmf, data):
    """Update the distribution of ability."""
    difficulty, outcome = data
    
    abilities = pmf.qs
    ps = prob_correct(abilities, difficulty)
    
    if outcome:
        pmf *= ps
    else:
        pmf *= 1 - ps
        
    pmf.normalize()

O "dado" é um tuple que contém a dificuldade de uma pergunta e o resultado: "Verdade" se a resposta foi correcta e "Falso" de outra forma.

Como teste, vamos fazer uma actualização com base nos resultados que simulámos anteriormente, com base numa pessoa com "capacidade=600" respondendo a 51 perguntas com "dificuldade=500".

In [62]:
actual_600 = prior.copy()

for outcome in outcomes:
    data = (500, outcome)
    update_ability(actual_600, data)

Eis como se parece a distribuição posterior.

In [63]:
actual_600.plot(color='C4')

decorate(xlabel='Ability',
         ylabel='PDF',
         title='Posterior distribution of ability')

A média posterior é bastante próxima da capacidade real do test-taker, que é 600.

In [64]:
actual_600.mean()

Se executarmos esta simulação novamente, obteremos resultados diferentes.

## Adaptação

Agora vamos simular um teste adaptativo.

Vou utilizar a seguinte função para escolher perguntas, começando com a estratégia mais simples: todas as perguntas têm a mesma dificuldade.

In [65]:
def choose(i, belief):
    """Choose the difficulty of the next question."""
    return 500

Como parâmetros, "escolha" toma "i", que é o índice da pergunta, e "crença", que é um "Pmf" que representa a distribuição posterior da "capacidade", com base nas respostas às perguntas anteriores.

Esta versão de "escolha" não utiliza estes parâmetros; eles estão lá para que possamos testar outras estratégias (ver os exercícios no final do capítulo).

A função seguinte simula uma pessoa a fazer um teste, dado que conhecemos a sua capacidade real.

In [66]:
def simulate_test(actual_ability):
    """Simulate a person taking a test."""
    belief = prior.copy()
    trace = pd.DataFrame(columns=['difficulty', 'outcome'])

    for i in range(num_questions):
        difficulty = choose(i, belief)
        outcome = play(actual_ability, difficulty)
        data = (difficulty, outcome)
        update_ability(belief, data)
        trace.loc[i] = difficulty, outcome
        
    return belief, trace

Os valores de retorno são um "Pmf" representando a distribuição posterior da capacidade e um "DataFrame" contendo a dificuldade das perguntas e os resultados.

Aqui está um exemplo, mais uma vez para um test-taker com 'ability=600'.

In [67]:
belief, trace = simulate_test(600)

Podemos usar o vestígio para ver quantas respostas estavam correctas.

In [68]:
trace['outcome'].sum()

E aqui está o aspecto do posterior.

In [69]:
belief.plot(color='C4', label='ability=600')

decorate(xlabel='Ability',
         ylabel='PDF',
         title='Posterior distribution of ability')

Mais uma vez, a distribuição posterior representa uma estimativa bastante boa da capacidade real do test-taker.

## Quantificando Precisão

Para quantificar a precisão das estimativas, vou utilizar o desvio padrão da distribuição posterior.  O desvio padrão mede a dispersão da distribuição, pelo que um valor mais elevado indica mais incerteza sobre a capacidade do test-taker.

No exemplo anterior, o desvio padrão da distribuição posterior é de cerca de 40.

In [70]:
belief.mean(), belief.std()

Para um exame em que todas as perguntas têm a mesma dificuldade, a precisão da estimativa depende fortemente da capacidade do test-taker.  Para mostrar isso, vou percorrer uma série de capacidades e simular um teste utilizando a versão de "escolha" que retorna sempre "dificuldade=500".

In [71]:
actual_abilities = np.linspace(200, 800)
results = pd.DataFrame(columns=['ability', 'posterior_std'])
series = pd.Series(index=actual_abilities, dtype=float, name='std')

for actual_ability in actual_abilities:
    belief, trace = simulate_test(actual_ability)
    series[actual_ability] = belief.std()

O gráfico seguinte mostra o desvio padrão da distribuição posterior para uma simulação em cada nível de capacidade.

Os resultados são ruidosos, pelo que também traço uma curva ajustada aos dados por [local regression](https://en.wikipedia.org/wiki/Local_regression).

In [72]:
from utils import plot_series_lowess

plot_series_lowess(series, 'C1')

decorate(xlabel='Actual ability',
         ylabel='Standard deviation of posterior')

O teste é mais preciso para pessoas com capacidades entre `500` e `600`, menos preciso para pessoas da gama alta, e ainda pior para pessoas da gama baixa.

Quando todas as perguntas têm dificuldade "500", uma pessoa com "capacidade=800" tem uma elevada probabilidade de acertá-las.  Portanto, quando o fazem, não aprendemos muito sobre elas.

Se o teste incluir perguntas com uma gama de dificuldades, fornece mais informações sobre pessoas nos extremos superior e inferior da gama.

Como exercício no final do capítulo, terá a oportunidade de experimentar outras estratégias, incluindo estratégias adaptativas que escolham cada questão com base em resultados anteriores.

## Poder Discriminatório

Na secção anterior utilizámos o desvio padrão da distribuição posterior para quantificar a precisão das estimativas.  Outra forma de descrever o desempenho do teste (em oposição ao desempenho dos testadores) é medir o "poder discriminatório", que é a capacidade do teste para distinguir correctamente entre testadores com diferentes capacidades.

Para medir o poder discriminatório, vou simular uma pessoa a fazer o teste 100 vezes; após cada simulação, vou usar a média da distribuição posterior como a sua "pontuação".

In [73]:
def sample_posterior(actual_ability, iters):
    """Simulate multiple tests and compute posterior means.
    
    actual_ability: number
    iters: number of simulated tests
    
    returns: array of scores
    """
    scores = []

    for i in range(iters):
        belief, trace = simulate_test(actual_ability)
        score = belief.mean()
        scores.append(score)
        
    return np.array(scores)

Aqui estão amostras de pontuações para pessoas com vários níveis de capacidade.

In [74]:
sample_500 = sample_posterior(500, iters=100)

In [75]:
sample_600 = sample_posterior(600, iters=100)

In [76]:
sample_700 = sample_posterior(700, iters=100)

In [77]:
sample_800 = sample_posterior(800, iters=100)

Eis como se apresentam as distribuições de pontuações.

In [78]:
from empiricaldist import Cdf

cdf_500 = Cdf.from_seq(sample_500)
cdf_600 = Cdf.from_seq(sample_600)
cdf_700 = Cdf.from_seq(sample_700)
cdf_800 = Cdf.from_seq(sample_800)

In [79]:
cdf_500.plot(label='ability=500', color='C1',
            linestyle='dashed')
cdf_600.plot(label='ability=600', color='C3')
cdf_700.plot(label='ability=700', color='C2',
            linestyle='dashed')
cdf_800.plot(label='ability=800', color='C0')

decorate(xlabel='Test score',
         ylabel='CDF',
         title='Sampling distribution of test scores')

Em média, as pessoas com maior capacidade obtêm pontuações mais elevadas, mas qualquer pessoa pode ter um dia mau, ou um dia bom, pelo que existe alguma sobreposição entre as distribuições.

Para pessoas com capacidades entre `500` e `600`, onde a precisão do teste é maior, o poder discriminatório do teste é também elevado.

Se as pessoas com capacidades `500` e `600` fizerem o teste, é quase certo que a pessoa com capacidades mais elevadas obterá uma pontuação mais alta.

In [80]:
np.mean(sample_600 > sample_500)

Entre pessoas com capacidades `600` e `700`, é menos certo.

In [81]:
np.mean(sample_700 > sample_600)

E entre as pessoas com capacidades `700` e `800`, não é de todo certo.

In [82]:
np.mean(sample_800 > sample_700)

Mas lembre-se que estes resultados são baseados num teste em que todas as questões são igualmente difíceis.

Se fizer os exercícios no final do capítulo, verá que o desempenho do teste é melhor se incluir perguntas com uma série de dificuldades, e ainda melhor se o teste for adaptável.

 Voltar atrás e modificar "escolha", que é a função que escolhe a dificuldade da próxima pergunta.

1. Escrever uma versão de "escolha" que devolve uma série de dificuldades utilizando "i" como um índice numa sequência de dificuldades.

2. Escreva uma versão de "escolha" que seja adaptável, de modo a escolher a dificuldade da próxima pergunta baseada na "crença", que é a distribuição posterior da capacidade do testador, com base no resultado das respostas anteriores.

Para ambas as novas versões, executar novamente as simulações para quantificar a precisão do teste e o seu poder discriminatório.

Para a primeira versão da "escolha", qual é a distribuição ideal das dificuldades?

Para a segunda versão, qual é a estratégia adaptativa que maximiza a precisão do teste em toda a gama de capacidades?

In [83]:
# Solution goes here

In [84]:
# Solution goes here